# Dancer Conflict Matrix Generator
This is to help create show order. Outputs directly onto Master Roster spreadsheet.

**BEFORE YOU START:**
1. make sure to share the master roster with ur scripts acc (example: hanjie@adt-scripts.iam.gserviceaccount.com)
2. make sure you have a client file (example: client_secret.json) in the directory

**TIMES TO RUN:**
1. After 1st showings (create show order for 2nd showings)
2. After 2nd showings (optional, if changes to show order are neccessary)

Ask choreogs to keep the **Master Roster up to date** before you run and that internal guest groups (e.g MIT Lion Dance) are part of the roster.

## Defaults
Edit values!

In [ ]:
# Orignal Code author: cindywang'22, Spring 2022
# Notebook author: j13'24, Fall 2023
# Last updated: j13'24, Fall 2023

client_ssh_name = 'client_secret.json'
spreadsheed_name = '[Fall 2023] Master Roster'

## Generator
boom boom pow

In [ ]:

import sys, subprocess
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'gspread'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'oauth2client'])
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import time
from time import strftime, localtime
import copy
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(client_ssh_name, scope)
client = gspread.authorize(creds)

# Find a workbook by name and open the first sheet
# Make sure you use the right name here.
spreadsheet = client.open(spreadsheed_name)

master_roster_sheet = spreadsheet.get_worksheet(0)
master_roster_list_of_lists = master_roster_sheet.get_all_values()

conflict_matrix_worksheet = spreadsheet.get_worksheet(1)
# print(master_roster_list_of_lists)

dances_start_ind = 7
dances_end_ind = len(master_roster_list_of_lists[0])
dances = master_roster_list_of_lists[0][dances_start_ind:dances_end_ind]
print(dances)



#sorry brain dead this is spaghetti code so sorry
conflict_matrix = [[d] for d in dances]
print(conflict_matrix)
print(len(dances))
none = [None] * len(dances)
print(none)
for row in conflict_matrix:
    row.extend([None]*len(dances))
header = ['']
header.extend(dances)
header = [header]
header.extend(conflict_matrix)
conflict_matrix = header
print(conflict_matrix)

dance_ind = {}
for dance in dances:
    dance_ind[dance] = conflict_matrix[0].index(dance)

first_dancer_row = 2
last_dancer_row = 247 # TODO UPDATE THIS EVERY TIME
for row in range(first_dancer_row,last_dancer_row-1):
    dancer = ''
    if master_roster_list_of_lists[row][1]:
        dancer = master_roster_list_of_lists[row][0] + ' ' + master_roster_list_of_lists[row][1] + ' ' + master_roster_list_of_lists[row][2]
    else:
        dancer = master_roster_list_of_lists[row][0] + ' ' + master_roster_list_of_lists[row][2]
        
    dances = []
        
    for col in range(dances_start_ind, dances_end_ind):
        if master_roster_list_of_lists[row][col].strip().lower() == 'x':
            dances.append(master_roster_list_of_lists[0][col])
    
    if len(dances) < 2:
        continue
    for i in range(len(dances)):
        for j in range(i+1, len(dances)):
            dance1_ind, dance2_ind = dance_ind[dances[i]], dance_ind[dances[j]]
            if dance1_ind < dance2_ind:
                dance1_ind, dance2_ind = dance2_ind, dance1_ind
                
            if not conflict_matrix[dance1_ind][dance2_ind]:
                conflict_matrix[dance1_ind][dance2_ind] = {
                    'dancers': [],
                    'num_conflicts': 0
                    }
            conflict_matrix[dance1_ind][dance2_ind]['dancers'].append(dancer)
            conflict_matrix[dance1_ind][dance2_ind]['num_conflicts']+=1
            
print(conflict_matrix)

conflict_matrix_copy = copy.deepcopy(conflict_matrix)

for r, row in enumerate(conflict_matrix):
    for c, entry in enumerate(row):
        if not entry:
            conflict_matrix[r][c] = ''
        else:
            if r > 0 and c > 0:
                conflict_matrix[r][c] = int(entry['num_conflicts'])
        

conflict_matrix_worksheet.update('A1:Z30',conflict_matrix)


for r, row in enumerate(conflict_matrix_copy):
    for c, entry in enumerate(row):
        if not entry:
            conflict_matrix_copy[r][c] = ''
        else:
            conflict_matrix_copy[r][c] = str(entry)
            
conflict_matrix_worksheet.update('A24:Z60',conflict_matrix_copy)